In [ ]:

# --- CONFIGURATION ---
FILE_PATH = 'gol
import pandas as pd
import numpy as np 
from prophet import Prophet 
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Set Matplotlib style for better visuals
plt.style.use('fivethirtyeight')
d_monthly_csv.csv' 
DATE_COLUMN = 'Date'    
VALUE_COLUMN = 'Price'  

print(f"Starting Time Series Project: {FILE_PATH}\n")

# 2. Data Loading
try:
    df = pd.read_csv(FILE_PATH)
    print(f"Data loaded successfully. Initial rows:\n{df.head()}")
except FileNotFoundError:
    print(f"Error: The file '{FILE_PATH}' was not found. Please check the path.")
    raise

# 3. Data Cleaning and Preprocessing 
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df = df.dropna()
df.set_index(DATE_COLUMN, inplace=True)

# *** THIS LINE CREATES THE df_resampled VARIABLE ***
df_resampled = df[VALUE_COLUMN].resample('M').mean().fillna(method='ffill').to_frame()

print(f"\nCleaned and resampled data shape: {df_resampled.shape}")
print("-" * 50)


df_prophet = df_resampled.reset_index()
df_prophet.rename(columns={DATE_COLUMN: 'ds', VALUE_COLUMN: 'y'}, inplace=True)

split_date = df_prophet['ds'].max() - pd.DateOffset(months=12) 
train_df = df_prophet[df_prophet['ds'] <= split_date]
test_df = df_prophet[df_prophet['ds'] > split_date]

print(f"Training set size: {len(train_df)} | Test set size: {len(test_df)}")
print("-" * 50)


model = Prophet(
    daily_seasonality=False,
    yearly_seasonality=True,
    weekly_seasonality=False,
    seasonality_mode='multiplicative' 
)
model.fit(train_df)
print("Prophet model trained successfully on the training data.")

future_dates = model.make_future_dataframe(periods=len(test_df), freq='M', include_history=False)
forecast = model.predict(future_dates)

performance_df = pd.merge(test_df, forecast[['ds', 'yhat']], on='ds', how='inner')
mae = mean_absolute_error(performance_df['y'], performance_df['yhat'])
rmse = np.sqrt(mean_squared_error(performance_df['y'], performance_df['yhat']))

print("\n--- Model Evaluation (Test Set) ---")
print(f"Mean Absolute Error (MAE): ${mae:.2f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")
print("-" * 50)


final_model = Prophet(
    daily_seasonality=False,
    yearly_seasonality=True,
    weekly_seasonality=False,
    seasonality_mode='multiplicative'
)
final_model.fit(df_prophet)

# Forecast for the next 12 months
future_12_months = final_model.make_future_dataframe(periods=12, freq='M')
final_forecast = final_model.predict(future_12_months)

print("Final 12-Month Gold Price Forecast Complete.")


tableau_export_df = final_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
tableau_export_df = pd.merge(tableau_export_df, df_prophet, on='ds', how='left')
tableau_export_df.rename(columns={'y': 'Actual_Price', 'yhat': 'Predicted_Price'}, inplace=True)
tableau_export_df.to_csv('gold_price_tableau_data.csv', index=False)

print("\nData exported successfully to 'gold_price_tableau_data.csv' for Tableau visualization.")


fig = final_model.plot(final_forecast)
plt.title("Gold Price Forecast with Prophet")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()